# 📚  LLM-as-judge Evaluation

LLM-as-a-Judge is a method where a Large Language Model is used not to generate answers, but to evaluate them. Instead of humans manually reviewing responses, the LLM acts like an automatic evaluator, scoring or critiquing outputs based on predefined criteria.

### 📌 Key Ideas

- Role reversal: The LLM is not producing content—it is judging content produced by another model or system.

- Objective evaluation: It scores responses for correctness, relevance, completeness, safety, or adherence to instructions.

- Prompt-based rules: The evaluation criteria are written in a judge prompt, turning the model into a consistent reviewer.

### 🧠 Why Use LLM-as-a-Judge

- Removes the need for manual evaluation at scale

- Provides consistent judgments compared to subjective human reviewers

- Works well for tasks like RAG evaluation, code correctness, summarization quality, and safety checks

### ⚙️ How It Works

Provide the judge LLM with:

- The prompt/question

- The model response you want to evaluate

- Ground truth (optional)

- The judge LLM analyzes the answer using evaluation instructions.

- It outputs structured feedback

### 🔍 Use Cases
| Area                                 | What It Judges                          |
| ------------------------------------ | --------------------------------------- |
| Retrieval Augmented Generation (RAG) | Faithfulness, relevance, hallucinations |
| Code Generation                      | Does generated code solve the problem?  |
| Summarization                        | Coverage, factual accuracy              |
| Chatbots                             | Safety, tone, harmful content           |
| Education                            | Automated grading of open text answers  |


This notebook implement LLM-as-judge in pure Python:

- Dataset: list of prompts (and optional reference answers)

- Target model call:
   prompt → target_model → candidate_answer

- Judge model call (LLM-as-judge):

- prompt + candidate_answer + (optional reference) → judge_model → JSON {score, label, reason}

- Collect results into a list / DataFrame.


In [6]:
import json
from typing import List, Dict, Optional

from dotenv import load_dotenv

from openai import OpenAI

load_dotenv()




True

### 2. Helper to call completions API

In [7]:
client = OpenAI()

def openai_chat(
    model_id: str,
    user_text: str,
    system_prompt: Optional[str] = None,
    temperature: float = 0.0,
) -> str:
    """
    Call an OpenAI chat model and return the text output.
    Works with GPT-4o, GPT-4o-mini, GPT-5-series, etc.
    """

    messages = []

    if system_prompt:
        messages.append({"role": "system", "content": system_prompt})

    messages.append({"role": "user", "content": user_text})

    response = client.chat.completions.create(
        model=model_id,
        messages=messages,
        temperature=temperature,
    )

    # Extract plain text from response
    return response.choices[0].message.content.strip()


### 3. Judge prompt template

In [8]:

JUDGE_SYSTEM_PROMPT = """
You are an expert evaluator for LLM outputs.
You strictly follow the scoring rubric and ALWAYS respond in valid JSON only.
Do not include any extra commentary outside of the JSON.
"""

def build_judge_user_prompt(
    instruction: str,
    model_response: str,
    reference_answer: Optional[str] = None,
) -> str:
    """
    Build the user message given to the judge model.
    The judge will score from 1 to 5 and explain briefly.
    """
    ref_section = (
        f"\n[Reference Answer]\n{reference_answer}\n"
        if reference_answer
        else "\n[Reference Answer]\nNone provided.\n"
    )

    return f"""
You are grading a model response.

[Instruction / User Prompt]
{instruction}

[Model Response]
{model_response}
{ref_section}

Scoring rubric (1–5):
- 1 = Completely wrong, irrelevant, or nonsensical.
- 2 = Major issues; partially answers but with serious problems.
- 3 = Acceptable but incomplete or with some issues.
- 4 = Good answer with minor issues.
- 5 = Excellent, complete, correct, and well-explained.

Return ONLY valid JSON in this exact schema:
{{
  "score": <number between 1 and 5>,
  "label": "<one of: poor, fair, good, very_good, excellent>",
  "reason": "<short explanation of why you gave this score>"
}}
"""


### 4. Judge call + JSON parsing

In [9]:

def judge_response(
    judge_model_id: str,
    instruction: str,
    model_response: str,
    reference_answer: Optional[str] = None,
) -> Dict:
    """
    Call the judge model and parse its JSON grading output.
    """
    user_prompt = build_judge_user_prompt(
        instruction=instruction,
        model_response=model_response,
        reference_answer=reference_answer,
    )

    raw_output = openai_chat(
        model_id=judge_model_id,
        user_text=user_prompt,
        system_prompt=JUDGE_SYSTEM_PROMPT,
        temperature=0.0,
    )

    # Try to parse JSON robustly (in case model adds whitespace, etc.)
    def extract_json(text: str) -> Dict:
        # naive but often enough: find first '{' and last '}'.
        start = text.find("{")
        end = text.rfind("}")
        if start == -1 or end == -1:
            raise ValueError(f"Judge output is not valid JSON:\n{text}")
        json_str = text[start : end + 1]
        return json.loads(json_str)

    result = extract_json(raw_output)

    # Basic sanity checks
    if "score" not in result or "reason" not in result:
        raise ValueError(f"JSON missing required keys: {result}")

    return result


### 5. (Optional) target model for generation

In [10]:

def generate_candidate(
    target_model_id: str,
    instruction: str,
    system_prompt: str = "You are a helpful assistant.",
) -> str:
    return openai_chat(
        model_id=target_model_id,
        user_text=instruction,
        system_prompt=system_prompt,
        temperature=0.0,
    )



### 6. Example dataset in memory

In [11]:
dataset: List[Dict] = [
    {
        "id": 1,
        "prompt": "Explain what a vector database does in two simple sentences.",
        "reference": "A vector database stores embeddings that represent text, images, or other data in high-dimensional vectors. It enables fast similarity search to find semantically related items.",
    },
    {
        "id": 2,
        "prompt": "List three benefits of using LLM-as-a-judge for model evaluation.",
        "reference": None,  # judge decides correctness based on reasoning quality
    },
    {
        "id": 3,
        "prompt": "Convert the sentence 'The model failed three times' into passive voice.",
        "reference": "Three failures were encountered by the model.",
    },
    {
        "id": 4,
        "prompt": "Summarize the following in one sentence: 'Reinforcement learning agents learn actions through rewards and penalties and improve performance over time.'",
        "reference": "Reinforcement learning agents optimize decisions by using feedback from rewards and penalties.",
    },
    {
        "id": 5,
        "prompt": "Give me 2 ways to reduce hallucinations in a RAG system.",
        "reference": "Use citation-based generation and restrict model answers to retrieved documents.",
    },
    {
        "id": 6,
        "prompt": "Is this statement true or false? 'FAISS is a text generation model.' Justify your answer.",
        "reference": "False. FAISS is a vector similarity search library, not a text generation model.",
    },
    {
        "id": 7,
        "prompt": "Write a single SQL query to fetch all employees earning more than 50,000.",
        "reference": "SELECT * FROM employees WHERE salary > 50000;",
    },
    {
        "id": 8,
        "prompt": "Provide a polite refusal to this request: 'Hack into my colleague’s email.'",
        "reference": "I can't help with hacking or accessing private accounts, but I can suggest legal ways to contact your colleague.",
    },
    {
        "id": 9,
        "prompt": "Explain 'prompt chaining' in less than 25 words.",
        "reference": "Prompt chaining breaks complex tasks into multiple linked prompts where each output becomes input for the next step.",
    },
    {
        "id": 10,
        "prompt": "Suggest three test metrics to evaluate a generative AI model.",
        "reference": None,  # judge model evaluates completeness + correctness
    },
]


### 7. Configuration: model IDs

In [12]:

# (Use GPT-4o / GPT-4o-mini / GPT-5 models as applicable)

TARGET_MODEL_ID = "gpt-4o-mini"     # fast, inexpensive model under test
JUDGE_MODEL_ID  = "gpt-4o"          # stronger model used as an evaluator (LLM-as-a-judge)

### 8. Full evaluation loop

In [14]:

results: List[Dict] = []

for item in dataset:
    prompt = item["prompt"]
    reference = item.get("reference")

    print(f"\n=== Evaluating item {item['id']} ===")
    print("Prompt:", prompt)

    # 1) Generate candidate answer from target model
    candidate = generate_candidate(
        target_model_id=TARGET_MODEL_ID,
        instruction=prompt,
    )
    print("Candidate response:\n", candidate)

    # 2) Judge the response
    grade = judge_response(
        judge_model_id=JUDGE_MODEL_ID,
        instruction=prompt,
        model_response=candidate,
        reference_answer=reference,
    )

    print("Judge result:", grade)

    results.append(
        {
            "id": item["id"],
            "prompt": prompt,
            "candidate": candidate,
            "score": grade["score"],
            "label": grade.get("label"),
            "reason": grade["reason"],
        }
    )




=== Evaluating item 1 ===
Prompt: Explain what a vector database does in two simple sentences.
Candidate response:
 A vector database stores and manages data in the form of high-dimensional vectors, which represent various types of information, such as text, images, or audio. It enables efficient similarity search and retrieval of data based on the proximity of these vectors in a multi-dimensional space, making it ideal for applications like recommendation systems and natural language processing.
Judge result: {'score': 5, 'label': 'excellent', 'reason': 'The model response accurately describes what a vector database does, mentioning both the storage of high-dimensional vectors and the efficient similarity search, aligning well with the reference answer.'}

=== Evaluating item 2 ===
Prompt: List three benefits of using LLM-as-a-judge for model evaluation.
Candidate response:
 Using LLMs (Large Language Models) as judges for model evaluation offers several benefits:

1. **Consistency a

### 9. Inspect results in Python

In [15]:

print("\n=== Summary ===")
for r in results:
    print(f"ID {r['id']}: score={r['score']} label={r['label']}")
    print("  Reason:", r["reason"])


=== Summary ===
ID 1: score=5 label=excellent
  Reason: The model response accurately describes what a vector database does, mentioning both the storage of high-dimensional vectors and the efficient similarity search, aligning well with the reference answer.
ID 2: score=5 label=excellent
  Reason: The response clearly and accurately lists three benefits of using LLMs as judges for model evaluation: consistency and objectivity, scalability, and rich feedback and insights. Each point is well-explained and relevant to the prompt, making the answer complete and correct.
ID 3: score=2 label=fair
  Reason: The model response attempts to convert the sentence into passive voice but incorrectly implies that the model was actively failed by something else, which changes the meaning. The structure is also awkward and not a direct passive transformation of the original sentence.
ID 4: score=4 label=good
  Reason: The model response captures the essence of reinforcement learning by mentioning perf